# Cadastro de Emissão

## Classe

In [26]:
import requests
import json

class Auth:
    def __init__(self, api_key, secret_key, base_url):
        self.api_key = api_key
        self.secret_key = secret_key
        self.token = None
        self.base_url = base_url

    def autenticar(self):
        url = "https://auth-frontier.sandbox.cloud.laqus.com.br/api/v1/auth"

        payload = json.dumps({
          "apiKey": self.api_key,
          "secretKey": self.secret_key
        })
        headers = {
          'accept': '*/*',
          'Content-Type': 'application/json'
        }

        response = requests.request("POST", url, headers=headers, data=payload)
        self.token = response.json()
        return self.token

class Solicitacao:
    def __init__(self, token, solicitacao_data, base_url):
        self.token = token
        self.solicitacao_data = solicitacao_data
        self.base_url = base_url

    def fazer_solicitacao(self):
        url = f"{self.base_url}/solicitacoes"
        headers = {
            'accept': 'application/json',
            'Content-Type': 'application/json',
            'Authorization': f'Bearer {self.token}'
        }
        response = requests.post(url, headers=headers, data=json.dumps(self.solicitacao_data))
        return response.json()

    def adicionar_informacoes_parcelas(self, id_operacao, informacoes_parcelas):
        url = f"{self.base_url}/solicitacoes/{id_operacao}/dados-financeiros"
        headers = {
            'accept': '*/*',
            'Content-Type': 'application/json',
            'Authorization': f'Bearer {self.token}'
        }
        response = requests.post(url, headers=headers, data=json.dumps(informacoes_parcelas))
        return response.json()

    def adicionar_parcelas(self, id_operacao, parcelas_data):
        url = f"{self.base_url}/solicitacoes/{id_operacao}/dados-financeiros/parcelas"
        headers = {
            'accept': '*/*',
            'Content-Type': 'application/json',
            'Authorization': f'Bearer {self.token}'
        }
        response = requests.post(url, headers=headers, data=json.dumps(parcelas_data))
        return response.json()

    def adicionar_documento(self, id_operacao, tipo_documento, arquivo_path):
        url = f"{self.base_url}/solicitacoes/{id_operacao}/documentos"
        payload = {'tipo': tipo_documento}
        files = [('arquivo', (arquivo_path.split('/')[-1], open(arquivo_path, 'rb'), 'application/pdf'))]
        headers = {
            'accept': 'application/json',
            'Authorization': f'Bearer {self.token}'
        }
        response = requests.post(url, headers=headers, data=payload, files=files)
        return response.json()

    def submeter_solicitacao(self, id_operacao):
        url = f"{self.base_url}/solicitacoes/{id_operacao}/submissao"
        headers = {
            'accept': 'application/json',
            'Authorization': f'Bearer {self.token}'
        }
        response = requests.post(url, headers=headers)
        return response.json()

## Carregar Arquivo



In [65]:
from google.colab import drive
import os
import pandas as pd
from datetime import datetime

In [66]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [67]:
def local_excel_dados_cadastro():
    pasta_arquivos = os.getcwd() + "/drive/MyDrive/Laqus"
    return pasta_arquivos

def local_pasta_arquivos():
    pasta_arquivos = os.getcwd() + "/drive/MyDrive/Laqus/Arquivos Emissão"
    return pasta_arquivos

In [68]:
dadosEmissao = pd.read_excel(local_excel_dados_cadastro() + "/Dados Emissão.xlsx",sheet_name="Post 1 - Solicitar Emissão",converters={'cnpjDoEmissor': str})

In [69]:
dadosEmissao

,numeroDaEmissao,numeroDaSerie,dataDaEmissao,tipo,dataDoVencimento,localDaEmissao,ufDaEmissao,cnpjDoEmissor,cnpjDoEscriturador,idOperacao
0,1ª,Única,2024-04-08,NotaComercial,2026-06-08,São Paulo,SP,64250001820484,86487909000131,NaN


In [70]:
dadosinvestidores = pd.read_excel(local_excel_dados_cadastro() + "/Dados Emissão.xlsx",sheet_name="Post 1 - Investidores",converters={'cpfCnpj': str,
                                                                                                                                      'cnpjDoCustodiante': str})

In [71]:
dadosFinanceiroInstrumento = pd.read_excel(local_excel_dados_cadastro() + "/Dados Emissão.xlsx",sheet_name="Post 2 - Info Financeira")

In [72]:
dadosParcelas = pd.read_excel(local_excel_dados_cadastro() + "/Dados Emissão.xlsx",sheet_name="Post 3 - Parcelas")

In [73]:
dadosEnderecoTermos = pd.read_excel(local_excel_dados_cadastro() + "/Dados Emissão.xlsx",sheet_name="Post 5 - AnexarTermo")

In [74]:
dadosEnderecoBoletim = pd.read_excel(local_excel_dados_cadastro() + "/Dados Emissão.xlsx",sheet_name="Post 6 - AnexarBoletim")

In [75]:
dadosEmissoesSubmetidas = pd.read_excel(local_excel_dados_cadastro() + "/Dados Emissão.xlsx",sheet_name="Post 7 - Submetidos")

## Executar

In [76]:
api_key = "c2FudG9zaW52ZXN0bWVudA=="
secret_key = "51313aa417b4f5c015f7805ab47e0db1d26012716de70a549268f02d64a67804f7da1fda8583522700d38e75376e5c6b1144cb4fa6cb89c095ad4a522a7646dc"
base_url = "https://emissao-frontier.sandbox.cloud.laqus.com.br/api/v1"

autenticacao = Auth(api_key, secret_key, base_url)
token_json = autenticacao.autenticar()
token = token_json['accessToken']

In [77]:
submetidos = []
problemas = []
dadosEmissoescomProblemas = pd.DataFrame()
for emissao in dadosEmissao.iterrows():
  df_emissao = pd.DataFrame(emissao[1])
  df_emissao_investidor = pd.merge(df_emissao.T, dadosinvestidores, on=["numeroDaEmissao", "numeroDaSerie", "dataDaEmissao"])
  df_emissao_financeiro = pd.merge(df_emissao.T, dadosFinanceiroInstrumento, on=["numeroDaEmissao", "numeroDaSerie", "dataDaEmissao"])
  df_emissao_parcelas = pd.merge(df_emissao.T, dadosParcelas, on=["numeroDaEmissao", "numeroDaSerie", "dataDaEmissao"])
  df_emissao_termo = pd.merge(df_emissao.T, dadosEnderecoTermos, on=["numeroDaEmissao", "numeroDaSerie", "dataDaEmissao"])
  df_emissao_boletim = pd.merge(df_emissao.T, dadosEnderecoBoletim, on=["numeroDaEmissao", "numeroDaSerie", "dataDaEmissao"])

  json_emissao = []
  json_investidores = []
  json_emissao_financeiro = []
  json_emissao_parcelas = []

  for _, row in df_emissao_investidor.iterrows():
    json_investidores.append({
        "cpfCnpj": str(row["cpfCnpj"]),
        "cnpjDoCustodiante": str(row["cnpjDoCustodiante"]),
        "precoUnitario": row["precoUnitario"],
        "quantidadeSubscrita": row["quantidadeSubscrita"]
    })
  json_emissao.append({
        "tipo": row["tipo"],
        "numeroDaEmissao": str(row["numeroDaEmissao"]),
        "numeroDaSerie": str(row["numeroDaSerie"]),
        "dataDaEmissao": row["dataDaEmissao"].strftime('%Y-%m-%d'),
        "dataDoVencimento":  row["dataDoVencimento"].strftime('%Y-%m-%d'),
        "localDaEmissao": row["localDaEmissao"],
        "ufDaEmissao": row["ufDaEmissao"],
        "cnpjDoEmissor": str(row["cnpjDoEmissor"]),
        "cnpjDoEscriturador": str(row["cnpjDoEscriturador"]),
        "investidores": json_investidores
    })
  solicitacao = Solicitacao(token, json_emissao[0], base_url)
  resposta_solicitacao = solicitacao.fazer_solicitacao()
  id_operacao = resposta_solicitacao['id']
  dadosEmissao.loc[(dadosEmissao["numeroDaEmissao"] == row["numeroDaEmissao"]) & (dadosEmissao["numeroDaSerie"]== row["numeroDaSerie"]) & (dadosEmissao["dataDaEmissao"] == row["dataDaEmissao"]),"idOperacao"] = id_operacao

  for _, row in df_emissao_financeiro.iterrows():
    json_emissao_financeiro.append({
         "custoDaEmissao": row["custoDaEmissao"],
          "valorNominalUnitario": row["valorNominalUnitario"],
          "quantidade": row["quantidade"],
          "percentualDosJurosRemuneratorios": row["percentualDosJurosRemuneratorios"],
          "percentualDoIndexador": float(row["percentualDoIndexador"]),
          "indexador": row["indexador"],
          "convencao": row["convencao"],
          "localDePagamento": row["localDePagamento"],
          "ufDePagamento": row["ufDePagamento"],
          "tipoDePagamento": row["tipoDePagamento"],
          "tabelaPrice": {
           "percentualSobre": row["percentualSobre"],
           "vencimentoDa1aParcela": row["vencimentoDa1aParcela"],
           "periodicidade": row["periodicidade"]
           }})

  resposta_info_financeira = solicitacao.adicionar_informacoes_parcelas(id_operacao, json_emissao_financeiro[0])

  dadosinvestidores.loc[(dadosinvestidores["numeroDaEmissao"] == row["numeroDaEmissao"]) & (dadosinvestidores["numeroDaSerie"]== row["numeroDaSerie"]) & (dadosinvestidores["dataDaEmissao"] == row["dataDaEmissao"]),"idOperacao"] = id_operacao

  dadosFinanceiroInstrumento.loc[(dadosFinanceiroInstrumento["numeroDaEmissao"] == row["numeroDaEmissao"]) & (dadosFinanceiroInstrumento["numeroDaSerie"]== row["numeroDaSerie"]) & (dadosFinanceiroInstrumento["dataDaEmissao"] == row["dataDaEmissao"]),"idOperacao"] = id_operacao


  parcelas = []
  for _, row in df_emissao_parcelas.iterrows():
    parcelas.append({
            "dataVencimento": row["dataVencimento"].strftime('%Y-%m-%d'),
            "valorTotal": row["valorTotal"],
            "valorAmortizacao": row["valorAmortizacao"]
        })
  json_emissao_parcelas.append({"parcelas":parcelas})

  resposta_parcelas = solicitacao.adicionar_parcelas(id_operacao, json_emissao_parcelas[0])

  dadosParcelas.loc[(dadosParcelas["numeroDaEmissao"] == row["numeroDaEmissao"]) & (dadosParcelas["numeroDaSerie"]== row["numeroDaSerie"]) & (dadosParcelas["dataDaEmissao"] == row["dataDaEmissao"]),"idOperacao"] = id_operacao


  for _, row in df_emissao_termo.iterrows():
    arquivo_path = row["raizLocalEnderecoDocumento"] + row["nomeArquivo"]
    tipo_documento = "TermoConstitutivoAssinado"
    resposta_documento1 = solicitacao.adicionar_documento(id_operacao, tipo_documento, arquivo_path)
    dadosEnderecoTermos.loc[(dadosEnderecoTermos["numeroDaEmissao"] == row["numeroDaEmissao"]) & (dadosEnderecoTermos["numeroDaSerie"]== row["numeroDaSerie"]) & (dadosEnderecoTermos["dataDaEmissao"] == row["dataDaEmissao"]),"idOperacao"] = id_operacao


  for _, row in df_emissao_boletim.iterrows():
    arquivo_path = row["raizLocalEnderecoDocumento"] + row["nomeArquivo"]
    tipo_documento = "BoletimSubscricaoAssinado"
    resposta_documento1 = solicitacao.adicionar_documento(id_operacao, tipo_documento, arquivo_path)
    dadosEnderecoBoletim.loc[(dadosEnderecoBoletim["numeroDaEmissao"] == row["numeroDaEmissao"]) & (dadosEnderecoBoletim["numeroDaSerie"]== row["numeroDaSerie"]) & (dadosEnderecoBoletim["dataDaEmissao"] == row["dataDaEmissao"]),"idOperacao"] = id_operacao


  resposta_submissao = solicitacao.submeter_solicitacao(id_operacao)


  try:
    status_subssao = resposta_submissao["status"]
  except:
    status_subssao = resposta_submissao["statusCode"]
    status = resposta_submissao["statusCode"]
    menssagem = resposta_submissao["message"]
    problemas.append({"Data":datetime.today().strftime('%Y-%m-%d'),
                       "menssagem":menssagem,
                       "status":status,
                       "numeroDaEmissao": str(row["numeroDaEmissao"]),
                       "numeroDaSerie": str(row["numeroDaSerie"]),
                       "dataDaEmissao": row["dataDaEmissao"],
                      })


  if status_subssao == 'Aguardando Liquidação':
    codigo_if = resposta_submissao["codigoInstrumentoFinanceiro"]
    status = resposta_submissao["status"]
    submetidos = {"idOperacao":id_operacao,
                       "Data":datetime.today().strftime('%Y-%m-%d'),
                       "codigoInstrumentoFinanceiro":codigo_if,
                       "status":status}
  dadosEmissoesSubmetidas.loc[len(dadosEmissoesSubmetidas)] = submetidos


dadosEmissoescomProblemas = pd.DataFrame(problemas)

In [50]:
solicitacao.submeter_solicitacao(id_operacao)

{'statusCode': 400,
 'message': '07179918067 investidor não existe ou investidor reprovado'}

In [78]:
dadosEmissoesSubmetidas

,idOperacao,Data,codigoInstrumentoFinanceiro,status
0,1094e813-de17-4443-acb4-672bc6618f9d,2024-04-29,LNC002400803,Aguardando Liquidação
